### Importing the dependencies

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


#### Data Collection and preprocessing

In [11]:
data = load_breast_cancer()
X = data.data
y = data.target

In [12]:
print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [13]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

##### Split the dataset into training set and test set

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [15]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


##### Standardize the data using standard scaler

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
type(X_train)

numpy.ndarray

In [18]:
#converting data into pytorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

### Neural Network Architecture

In [21]:
#define the nn architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [22]:
# define hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [23]:
#initialize the neural network
model = NeuralNet(input_size,hidden_size,output_size).to(device)

In [24]:
#define the loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#### Training the neural network

In [25]:
#training the model 
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1,1))
    loss.backward()
    optimizer.step()


    # calculate the accuracy 
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == y_train.view(-1,1)).float().sum()
        accuracy = correct/y_train.size(0)

    if (epoch+1)%10 == 0:
        print(f"Epoch : [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy : {accuracy.item()*100:.2f}%")

Epoch : [10/100], Loss : 0.5818, Accuracy : 87.47%
Epoch : [20/100], Loss : 0.4476, Accuracy : 92.53%
Epoch : [30/100], Loss : 0.3462, Accuracy : 92.97%
Epoch : [40/100], Loss : 0.2730, Accuracy : 93.85%
Epoch : [50/100], Loss : 0.2220, Accuracy : 94.51%
Epoch : [60/100], Loss : 0.1866, Accuracy : 95.38%
Epoch : [70/100], Loss : 0.1613, Accuracy : 95.82%
Epoch : [80/100], Loss : 0.1426, Accuracy : 96.70%
Epoch : [90/100], Loss : 0.1282, Accuracy : 97.14%
Epoch : [100/100], Loss : 0.1168, Accuracy : 97.80%


### Model evaluation

In [26]:
#evaluation on training set
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)
    print(f"Accuracy on training data: {accuracy.item()*100:.2f}%")

Accuracy on training data: 97.80%


In [27]:
#evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == y_test.view(-1,1)).float().sum()
    accuracy = correct/y_test.size(0)
    print(f"Accuracy on training data: {accuracy.item()*100:.2f}%")

Accuracy on training data: 96.49%
